## Kodkväll-AI

In [1]:
#imports

from torch import nn
import torch
import gym
from collections import deque
import itertools
import numpy as np
import random

#hyperparams

gamma = .99
batch_size = 32
buffer_size = 50000
min_replay_size = 1000
eps_start = 1.0
eps_end = 0.02
eps_decay_steps = 10000
target_update_freq = 1000
learning_rate = 5e-4

### Miljö

Simulatorn kommer från Open AI och går att läsa mer om här: [Open AIs simulator](https://gym.openai.com/envs/CartPole-v0)

*En pole är fäst på en släde, som rör sig längsmed en friktions fri bana. Man kan påvreka polen genom att ge den en kraft av +1 eller -1. Simulereingen börjar med att polen står rakt upp och målet är att den inte ska falla. För verje tidssteg som pole klarar sig ges +1 poäng. Episoden avslutas om polen avviker mer än 2.4 enheter från center eller om dess vinkel är mer än 15 grader från en vertikal position. Episoden avslutat automatisk efter 200 steg.*


![title](./cartpole.jpg)

In [ ]:
#Render env

env = gym.make('CartPole-v0')
env.reset()

while True:
    action = env.action_space.sample()
    obs, _, done, _ = env.step(action)
    env.render()
    if done:
        env.reset()

### Utforskning

för att agenten ska kuna utforska men även utnyttja tidigar lärdommar behövs en algorithm som tar hänsyn till båda dess aspekter. Här behöver vi implementera Epsilon-greedy algorithmen som tillåter detta:


In [ ]:
def epsilon_greedy(eps):
    '''
    Beräknar en action enligt epsilon-greedy policyn.
    Inputs:
        eps    - utforskningsparameter.
    Returns:
        action     - Vald action (0 eller 1, med andra ord, vänster eller höger), Integer.
    '''
    #Code here
    
    

### Neuralt nätverk för Q-learning

In [ ]:
class Network(nn.Module):
    '''
    Neuralt nätverk bestående av
    Inputs:
        eps    - utforskningsparameter.
    Returns:
        action     - Vald action (0 eller 1, med andra ord, vänster eller höger), Integer.
    '''
    def __init__(self, env):
        super().__init__()
        
        #Create neural net structure here and bind to self.net
        
    
    def forward(self, x):
        return self.net(x)

    def act(self, obs):
        obs_t = torch.as_tensor(obs, dtype=torch.float32)
        q_values = self(obs_t.unsqueeze(0))

        max_q_index = torch.argmax(q_values, dim=1)[0]
        action = max_q_index.detach().item()
        return action

### Initialisering av miljö och buffer

In [ ]:

env = gym.make('CartPole-v0')
replay_buffer = deque(maxlen=buffer_size)
rew_buffer = deque([0,0], maxlen=100)

episode_reward = 0.0

online_net = Network(env)
target_net = Network(env)

target_net.load_state_dict(online_net.state_dict())
optimizer = torch.optim.Adam(online_net.parameters(), lr=learning_rate)

#Init buffer
obs = env.reset()

for _ in range(min_replay_size):
    action = env.action_space.sample()

    new_obs, reward, done, _ = env.step(action)
    trans = (obs, action, reward, done, new_obs)
    replay_buffer.append(trans)
    obs = new_obs

    if done:
        obs = env.reset()

### Träning av agent

In [ ]:
obs = env.reset()

for step in itertools.count():
    eps = np.interp(step, [0, eps_decay_steps], [eps_start, eps_end])

    action = epsilon_greedy(eps)
    
    new_obs, reward, done, _ = env.step(action)
    
    trans = (obs, action, reward, done, new_obs)
    replay_buffer.append(trans)
    obs = new_obs
    
    episode_reward += reward
    
    if done:
        obs = env.reset()

        rew_buffer.append(episode_reward)
        episode_reward = 0.0

    #after solved, show env
    if len(rew_buffer) >= 100:
        if np.mean(rew_buffer) >= 195:
            while True:
                action = online_net.act(obs)
                obs, _, done, _ = env.step(action)
                env.render()
                if done:
                    env.reset()

    #gradient step
    transitions = random.sample(replay_buffer, batch_size)

    obses = np.asanyarray([t[0] for t in transitions])
    actions = np.asarray([t[1] for t in transitions])
    rews = np.asarray([t[2] for t in transitions])
    dones = np.asarray([t[3] for t in transitions])
    new_obses = np.asanyarray([t[4] for t in transitions])

    obses_t = torch.as_tensor(obses, dtype=torch.float32)
    actions_t = torch.as_tensor(actions, dtype=torch.int64).unsqueeze(-1)
    rews_t = torch.as_tensor(rews, dtype=torch.float32).unsqueeze(-1)
    dones_t = torch.as_tensor(dones, dtype=torch.float32).unsqueeze(-1)
    new_obses_t = torch.as_tensor(new_obses, dtype=torch.float32)

    #compute targets
    target_q_values = target_net(new_obses_t)
    max_target_q_values = target_q_values.max(dim=1, keepdims=True)[0]

    targets = rews_t + gamma * (1 - dones_t) * max_target_q_values
    
    #compute loss
    q_values = online_net(obses_t)

    action_q_values = torch.gather(q_values, dim=1, index=actions_t)

    loss = nn.functional.smooth_l1_loss(action_q_values, targets)

    #gradient decent
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #update target net
    if step % target_update_freq == 0:
        target_net.load_state_dict(online_net.state_dict())

    #logging
    if step % 1000 == 0:
        print()
        print('step: {}'.format(step))
        print('average reward: {}'.format(np.mean(rew_buffer)))